In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import entropy
import math as math
import seaborn as sns
from matplotlib.colors import LogNorm
%pylab inline

!pip install sociophysicsdatahandler

Populating the interactive namespace from numpy and matplotlib


In [4]:
from sociophysicsDataHandler import SociophysicsDataHandler
!cat auth.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
dh = SociophysicsDataHandler()
pd.set_option('float_format', '{:f}'.format)

Login error. 
HTTP error: 401


In [6]:
# dates
list_days = ['20210401','20210402','20210403','20210404','20210405','20210406','20210407','20210408','20210409','20210410']

In [7]:
# concat multiple dates worth of data in one large dataframe
df_list = []
for i in list_days:
  try:
    file_path = 'ehv/platform2.1' + '/' + i + '/'
    flst = dh.list_files(file_path)

    dh.fetch_prorail_data_from_path(file_path + '/' + flst.name[6])
    df_list.append(dh.df)
  except:
    pass
large_dataframe = pd.concat(df_list)

targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210401/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210402/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210403/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210404/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210405/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210406/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210407/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210408/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210409/
targeting path /ProRail_USE_LL_data\ehv/platform2.1/20210410/


ValueError: No objects to concatenate

In [ ]:
# sort by tracked object and use central difference formula to compute velocity
large_dataframe = large_dataframe.sort_values(by=['tracked_object'])
large_dataframe['x_velocity']=np.gradient(large_dataframe['x_pos'])
large_dataframe['y_velocity']=np.gradient(large_dataframe['y_pos'])
large_dataframe['velocity'] = np.sqrt(large_dataframe['x_velocity']**2 + large_dataframe['y_velocity']**2)

In [ ]:
# choose waiters
waiters = large_dataframe[(large_dataframe['velocity']<10) & (large_dataframe['y_pos']<70000)]

In [ ]:
# create heatmap to see where people like to wait
H, xedges, yedges = histogram2d(waiters.y_pos,waiters.x_pos,bins=(150,30))
H = H.T

In [ ]:
# remove unwanted datapoints such as the high values at the stairs,
# h is now the probability landscape
H[16:21,127:138]=10
h = H[7:-1]

In [ ]:
fig, ax = subplots(figsize=(22, 3))
ax.imshow(h,cmap = 'Blues',norm = LogNorm())
tight_layout()

idea: to make this time dependent, use the heat equation to average the probability landscape when the train arrives.